In [58]:
import os
import time
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import matplotlib.pyplot as plt
from model import keras_models
from module_DTI import my_dataset

In [51]:
dict_directories = {'dir_ROOT': 'C:\\Users\\csjeong\\Desktop\\research\\dataset\\conv_DTI\\2012, Tabei', }
dict_directories.update({
    'DTI_adjmat' : dict_directories['dir_ROOT'] + '\\inter_admat.txt',
    'drug' : dict_directories['dir_ROOT'] + '\\drug_repmat.txt',
    'target' : dict_directories['dir_ROOT']+ '\\target_repmat.txt'
})
DTI=my_dataset.DTI_Dataset(dict_directories)
DTI.load_data()
DTI.load_pos_neg_samples(neg_to_pos_ratio=1)
DTI.split_train_test_set(split_ratio=0.99)

config = tf.ConfigProto(log_device_placement=True)
n_class = 2
feature_size = DTI.train.data.shape[1]
n_train = DTI.train.data.shape[0]
n_test = DTI.test.data.shape[0]
learning_rate = 0.001
batch_size = 100
training_iteration = 2000

avg_cost_list = []
train_x, train_y = DTI.train.next_batch(batch_size=n_train, pos_neg_label=DTI.pos_neg_label)
test_x, test_y = DTI.test.next_batch(DTI.test.data.shape[0], DTI.pos_neg_label, one_hot_encoding=True)
print('epoch:\t' + str(epoch))


# of drug: 1862
# of target: 1554

# of drugs: 1862
# of features of a drug: 881

# of targets: 1554
# of features of a target: 876

# of drug-target interactions: 4809

Load existing positive & negative sample files.
# of positive samples: 4809
# of features of a positive sample: 1757

# of negative samples: 4809
# of features of a negative sample: 1757

epoch:	5


In [56]:
class MyConfig(object):
    learning_rate = 0.001
    batch_size = 100
    training_iteration = 200
    epoch = int(training_iteration * batch_size / DTI.train.data.shape[0])
    keep_prob=1.0
    feature_size=1757
    n_class=2



In [62]:
class MyTensorflow(object):
    def __init__(self, session, config, is_training=False):
        self.config=config
        self._session = session
        self._batch_size = config.batch_size
        self._training_iteration = config.training_iteration
        self._learning_rate = config.learning_rate
        self._input_size = [None, config.feature_size]
        self._output_size = [None, config.n_class]
           
    def train(self, DTI):
        avg_cost = 0.
        batch_xs, batch_ys = DTI.train.next_batch(self._batch_size, DTI.pos_neg_label)
        self._session.run(self._optimize, feed_dict={self._x: batch_xs, self._y: batch_ys})
        avg_cost += self._session.run(self._loss, feed_dict={self._x: batch_xs, self._y: batch_ys}) / self._batch_size
        return avg_cost

    def get_accuracy(self, DTI):
        test_x, test_y = DTI.test.next_batch(DTI.test.data.shape[0], DTI.pos_neg_label, one_hot_encoding=True)
        ac = self._session.run(self._accuracy, feed_dict={self._x: test_x, self._y: test_y})
        return ac
     
class mymodel(MyTensorflow):
    def __init__(self, session, config, is_training=False):
        super(mymodel,self).__init__(session,config,is_training)
        self.build_graph()
        
    def loss(self,true_logits, predicted_logits):
        return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=predicted_logits, labels=true_logits))
    
    def optimize(self,loss, optimizer=tf.train.AdamOptimizer()):
        optimizer.learning_rate=self._learning_rate
        return optimizer.minimize(loss)
    
    def build_graph(self):
        x = tf.placeholder(tf.float32, self._input_size)
        y = tf.placeholder(tf.float32, self._output_size)
        W = tf.Variable(tf.zeros([self.config.feature_size, self.config.n_class]), name='W')
        b = tf.Variable(tf.zeros([n_class]), name='b')
        y_pred = tf.nn.softmax(tf.matmul(x, W) + b)
        
        _loss = self.loss(y,y_pred)
        _optimize=self.optimize(_loss,optimizer=tf.train.AdamOptimizer())
        

        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_pred, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype='float32'))
    
        self._x=x
        self._y=y
        self._W=W
        self._b=b
        self._y_pred=y_pred
        self._loss=_loss
        self._optimize=_optimize
        self._correct_prediction=correct_prediction
        self._accuracy=accuracy
        tf.global_variables_initializer().run()
        self.saver=tf.train.Saver()


my_config = MyConfig()
with tf.Graph().as_default(), tf.Session() as session:
    m = mymodel(session, my_config)
    for _ in range(my_config.training_iteration):
        avg_cost = m.train(DTI)
    #     print(avg_cost)
    print(m.get_accuracy(DTI))
    print(my_config.epoch)
    m.saver.save(session,
                    os.path.join('./saved_model', "tf_model_test.ckpt"))
    


0.6530612
2


In [ ]:
epoch = 20
total_data=3000

batch_size=100
iteration=600

batch_size=101
iteration=594 + 1
